In [1]:
!ls

answerquestion.txt  followMLdata.csv   quesAnstab.txt  word2vec_model
edit		    followMLdata.xlsx  sample_data


In [0]:
#import all the necessary packages.

from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
#import plotly
#import plotly.figure_factory as ff
#from plotly.graph_objs import Scatter, Layout


In [0]:
data = pd.read_csv('followMLdata.csv')
data['question'].head()

0                            Are you a self-motivator?
1    What matters to you more - job satisfaction or...
2    What matters to you more - job satisfaction or...
3    Have you worked with someone unprofessional, h...
4    Have you worked with someone unprofessional, h...
Name: question, dtype: object

In [0]:
data.columns

Index(['Unnamed: 0', 'question', 'answer', 'followUp'], dtype='object')

In [0]:
data.shape

(1086, 4)

In [0]:
data['answer'].describe()

count                                                  1086
unique                                                  465
top       Once we had to organize an event in our colleg...
freq                                                     10
Name: answer, dtype: object

In [0]:
# we use the list of stop words that are downloaded from nltk lib.
stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)

def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            # remove the special chars in review like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalnum()))
            # Conver all letters to lower-case
            word = word.lower()
            # stop-word removal
            if not word in stop_words:
                string += word + " "
        data[column][index] = string

LookupError: ignored

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
qa = data[['question','answer']]
target = data[['followUp']]


In [0]:

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

In [0]:
with open('word2vec_model', 'rb') as handle:
    model = pickle.load(handle)

In [0]:
def get_word_vec(sentence, doc_id, m_name):
    # sentence : title of the apparel
    # doc_id: document id in our corpus
    # m_name: model information it will take two values
        # if  m_name == 'avg', we will append the model[i], w2v representation of word i
        # if m_name == 'weighted', we will multiply each w2v[word] with the idf(word)
    vec = []
    for i in sentence.split():
        if i in vocab:
            if m_name == 'weighted' and i in  idf_title_vectorizer.vocabulary_:
                vec.append(idf_title_features[doc_id, idf_title_vectorizer.vocabulary_[i]] * model[i])
            elif m_name == 'avg':
                vec.append(model[i])
        else:
            # if the word in our courpus is not there in the google word2vec corpus, we are just ignoring it
            vec.append(np.zeros(shape=(300,)))
    # we will return a numpy array of shape (#number of words in title * 300 ) 300 = len(w2v_model[word])
    # each row represents the word2vec representation of each word (weighted/avg) in given sentance 
    return  np.array(vec)

In [0]:
def get_distance(vec1, vec2):
    # vec1 = np.array(#number_of_words_title1 * 300), each row is a vector of length 300 corresponds to each word in give title
    # vec2 = np.array(#number_of_words_title2 * 300), each row is a vector of length 300 corresponds to each word in give title
    
    final_dist = []
    # for each vector in vec1 we caluclate the distance(euclidean) to all vectors in vec2
    for i in vec1:
        dist = []
        for j in vec2:
            # np.linalg.norm(i-j) will result the euclidean distance between vectors i, j
            dist.append(np.linalg.norm(i-j))
        final_dist.append(np.array(dist))
    # final_dist = np.array(#number of words in title1 * #number of words in title2)
    # final_dist[i,j] = euclidean distance between vectors i, j
    return np.array(final_dist)



In [0]:
vocab = model.keys()
# this function will add the vectors of each word and returns the avg vector of given sentance
def build_avg_vec(sentence, num_features, doc_id, m_name):
    # sentace: its title of the apparel
    # num_features: the lenght of word2vec vector, its values = 300
    # m_name: model information it will take two values
        # if  m_name == 'avg', we will append the model[i], w2v representation of word i
        # if m_name == 'weighted', we will multiply each w2v[word] with the idf(word)

    featureVec = np.zeros((num_features,), dtype="float32")
    # we will intialize a vector of size 300 with all zeros
    # we add each word2vec(wordi) to this fetureVec
    nwords = 0
    
    for word in sentence.split():
        nwords += 1
        if word in vocab:
            if m_name == 'weighted' and word in  idf_title_vectorizer.vocabulary_:
                featureVec = np.add(featureVec, idf_title_features[doc_id, idf_title_vectorizer.vocabulary_[word]] * model[word])
            elif m_name == 'avg':
                featureVec = np.add(featureVec, model[word])
    if(nwords>0):
        featureVec = np.divide(featureVec, nwords)
    # returns the avg vector of given sentance, its of shape (1, 300)
    return featureVec

In [0]:
f=open('quesAnstab.txt','w+')
list1=[]
for i in range(len(qa)):
  list1.append((qa['question'][i]+'\t'+qa['answer'][i]))
 # print(str(i)+"---->"+list1[i])
  f.write(list1[i]+'\n')
f.close()  

In [0]:
!ls
!cat quesAnstab.txt

answerquestion.txt  followMLdata.csv   quesAnstab.txt  word2vec_model
edit		    followMLdata.xlsx  sample_data
Are you a self-motivator?	Absolutely. For me, internal motivation works far more than external motivation ever could. Yes, at first, it may seem like I want some sort of external motivation, but the very end, my heart goes into the work assigned only when my own self pushes me to do it.
What matters to you more - job satisfaction or salary?	According to me, job satisfaction covers all - the quality and quantity of work, salary, company environment and others. Yes, cooperation and adjustment is needed in a company. 
What matters to you more - job satisfaction or salary?	According to me, job satisfaction covers all - the quality and quantity of work, salary, company environment and others. Yes, cooperation and adjustment is needed in a company. 
Have you worked with someone unprofessional, how did you handle it?	During my B-Tech final semester internship, I have experienced unpr

In [0]:
doc_id = 0
w2v_title = []
# for every title we build a avg vector representation
for i in list1:
    #print(i , build_avg_vec(i, 300, doc_id,'avg'))
    w2v_title.append(build_avg_vec(i, 300, doc_id,'avg'))
    doc_id += 1

# w2v_title = np.array(# number of doc in courpus * 300), each row corresponds to a doc 

w2v_title = np.array(w2v_title) 
print(w2v_title.shape)

(1086, 300)


'Are you a self-motivator?Absolutely. For me, internal motivation works far more than external motivation ever could. Yes, at first, it may seem like I want some sort of external motivation, but the very end, my heart goes into the work assigned only when my own self pushes me to do it.'